## Loading JSON

In [1]:
import pandas as pd
import json

def load_data(location):
    f = open(location)
    data = json.load(f)
    
    arr = []
    
    for key in data:
        temp = []

        if data[key]['edges'] == False:
            temp.append(0)
        else:
            temp.append(1)
        if data[key]['magnitude'] == False:
            temp.append(0)
        else:
            temp.append(1)
        temp.append(data[key]['cascadeClassifier']['bottom_left'])
        temp.append(data[key]['cascadeClassifier']['bottom_right'])
        
        arr.append(temp)

    return pd.DataFrame(arr, columns=['edges', 'magnitude', 'quadrant3', 'quadrant4'])


data0026 = load_data('./jsons/2011_09_29_drive_0026_extract.json')
data0048 = load_data('./jsons/2011_09_26_drive_0048_extract.json')
data0113 = load_data('./jsons/2011_09_26_drive_0113_extract.json')

print(data0113)

    edges  magnitude  quadrant3  quadrant4
0       0          0          0          0
1       0          0          0          0
2       0          0          0          1
3       0          0          0          0
4       0          0          0          0
..    ...        ...        ...        ...
88      1          0          0          0
89      1          1          0          0
90      1          1          0          0
91      1          1          0          0
92      1          1          0          0

[93 rows x 4 columns]


## Loading labeld results

In [2]:
result0026 = pd.read_csv('./utils/2011_09_29_drive_0026_extract.csv')
result0048 = pd.read_csv('./utils/2011_09_26_drive_0048_extract.csv')
result0113 = pd.read_csv('./utils/2011_09_26_drive_0113_extract.csv')

print(result0113)

             frame  safety
0   0000000000.png       0
1   0000000001.png       0
2   0000000002.png       0
3   0000000003.png       0
4   0000000004.png       0
..             ...     ...
88  0000000088.png       1
89  0000000089.png       1
90  0000000090.png       1
91  0000000091.png       1
92  0000000092.png       1

[93 rows x 2 columns]


---

## Trainning

In [3]:
from sklearn.neural_network import MLPClassifier

In [4]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)

X = data0026
y = result0026['safety']

clf.fit(X, y)

X = data0048
y = result0048['safety']

clf.fit(X, y)


# Leave data0113 for checking accuracy
# X = data0113
# y = result0113['safety']

# clf.fit(X, y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

Save the trained model.

In [5]:
import pickle

with open('model.pkl','wb') as f:
    pickle.dump(clf, f)

## Results

In [6]:
X = data0113

y_act = result0113['safety']
y_pred = clf.predict(X)

In [7]:
from prettytable import PrettyTable

def calculate_metrics(true_labels, pred_labels):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for i in range(len(true_labels)):
        if true_labels[i] == pred_labels[i]:
            if true_labels[i] == 1:
                tp += 1
            else:
                tn += 1
        else:
            if true_labels[i] == 1:
                fn += 1
            else:
                fp += 1
    
    accuracy = (tp + tn) / len(true_labels)
    false_positive_rate = fp / (fp + tn)
    false_negative_rate = fn / (fn + tp)
    
    # create a table to display the results
    x = PrettyTable()
    x.field_names = ["Metric", "Value"]
    x.add_row(["Accuracy", "{:.2f}".format(accuracy)])
    x.add_row(["False Positive Rate", "{:.2f}".format(false_positive_rate)])
    x.add_row(["False Negative Rate", "{:.2f}".format(false_negative_rate)])
    
    print(x)

In [8]:
calculate_metrics(y_act, y_pred)

+---------------------+-------+
|        Metric       | Value |
+---------------------+-------+
|       Accuracy      |  0.60 |
| False Positive Rate |  0.25 |
| False Negative Rate |  0.61 |
+---------------------+-------+
